In [1]:
import requests
import json
import time
import pandas as pd
from tqdm import tqdm

weibo_total_comments_path = "../result/task3/weibo_total_comments.csv"
bilibili_total_comments_path = "../result/task3/bilibili_total_comments.csv"
weibo_df = pd.read_csv(weibo_total_comments_path)
bilibili_df = pd.read_csv(bilibili_total_comments_path)

output_path = "../result/task6"

In [2]:

wb = weibo_df[['content', 'time']]  

wb.head(10)  # 展示数据的前10行

,content,time
0,代肝dd哦,2025-03-12 00:16:25
1,转发微博,2025-03-09 17:53:38
2,加强弑君者,2025-03-08 11:02:23
3,加强近卫阿米娅 加强医疗阿米娅 加强能天使 加强Ash 加强远牙 加强霍尔海雅 加强荒芜拉普...,2025-03-08 09:47:36
4,朋友们看看视频的右下角那行字呢……yj是有权利落地前修改数值的……当然这不是把其他干员做得稀...,2025-03-07 20:47:59
5,请加强霍尔海雅,2025-03-07 20:11:07
6,悄悄改角色是吧，你他妈知道你做了一坨啊,2025-03-07 16:04:12
7,回复@023-纯真chan:vvan其实真没啥人气的，至少长夜临光后到崔金前，我个人只看到过...,2025-03-07 15:53:12
8,回复@023-纯真chan:vvan其实真没啥人气的，至少长夜临光后到崔金前，我个人只看到过...,2025-03-07 15:53:12
9,凭什么只改香蕉姐！！！重做君君！,2025-03-07 13:30:14


In [3]:
client_id = ''  # API key
client_secret = ''  # secret key
res = requests.post(f'https://aip.baidubce.com/oauth/2.0/token?'
                    f'grant_type=client_credentials'
                    f'&client_id={client_id}'
                    f'&client_secret={client_secret}')  # 进行鉴权认证
access_token = res.json()['access_token']  # 获得access_token

In [4]:
# 使用access_token构造url
url = f'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token={access_token}'
wb['positive_prob'] = 0  # 添加一列来记录正向情感得分
wb['negative_prob'] = 0  # 添加一列来记录负向情感得分
wb.head(10)  # 展示数据的前10行

,content,time,positive_prob,negative_prob
0,代肝dd哦,2025-03-12 00:16:25,0,0
1,转发微博,2025-03-09 17:53:38,0,0
2,加强弑君者,2025-03-08 11:02:23,0,0
3,加强近卫阿米娅 加强医疗阿米娅 加强能天使 加强Ash 加强远牙 加强霍尔海雅 加强荒芜拉普...,2025-03-08 09:47:36,0,0
4,朋友们看看视频的右下角那行字呢……yj是有权利落地前修改数值的……当然这不是把其他干员做得稀...,2025-03-07 20:47:59,0,0
5,请加强霍尔海雅,2025-03-07 20:11:07,0,0
6,悄悄改角色是吧，你他妈知道你做了一坨啊,2025-03-07 16:04:12,0,0
7,回复@023-纯真chan:vvan其实真没啥人气的，至少长夜临光后到崔金前，我个人只看到过...,2025-03-07 15:53:12,0,0
8,回复@023-纯真chan:vvan其实真没啥人气的，至少长夜临光后到崔金前，我个人只看到过...,2025-03-07 15:53:12,0,0
9,凭什么只改香蕉姐！！！重做君君！,2025-03-07 13:30:14,0,0


In [5]:
print(wb['content'][1])

转发微博


In [8]:
# 遍历所有的评论
for i in tqdm(range(0, 5)):
    data = json.dumps({'text': wb['content'][i]})  # 构建请求的data
    res = requests.post(url, data).json()  # 请求API数据
    if 'items' in res.keys():
        wb.loc[i, 'positive_prob'] = res['items'][0]['positive_prob']  # 记录正向得分
        wb.loc[i, 'negative_prob'] = res['items'][0]['negative_prob']  # 记录负向得分
    time.sleep(1)  # 间隔0.5秒
    
# 将结果存储为csv文件
wb.to_csv(f'{output_path}/wbemotion.csv', index=False)

  0%|          | 0/5 [00:00<?, ?it/s]C:\Users\s6_ti\AppData\Local\Temp\ipykernel_35328\3279492459.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.958046' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  wb.loc[i, 'positive_prob'] = res['items'][0]['positive_prob']  # 记录正向得分
C:\Users\s6_ti\AppData\Local\Temp\ipykernel_35328\3279492459.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0419543' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  wb.loc[i, 'negative_prob'] = res['items'][0]['negative_prob']  # 记录负向得分
100%|██████████| 5/5 [00:11<00:00,  2.30s/it]
